In [1]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?


Answer: """

In [3]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [4]:
from langchain.llms import OpenAI

openai = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key = os.environ["OPENAI_API_KEY"]
)

In [5]:
print(openai(prompt))


Hugging Face's `transformers` library, OpenAI's `openai` library, and Cohere's `cohere` library all offer LLMs.


### Few-shot prompting

In [6]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty. Here are some examples: 

User: Who you gonna call?
AI: """

openai.temperature = 1.0 #makes it more probabilistic so that nucleus sampling is used instead of greedy decoding

print(openai(prompt))

 I'm an AI, not a ghostbuster. I don't have a proton pack ready to go.


Pretty good, but we could improve it even further by training the model on a few examples provided in the prompt. i.e., few-shot learning.

In [7]:
from langchain import FewShotPromptTemplate, PromptTemplate

examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables = ["query", "answer"],
    template = example_template
)

prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: 
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n"
)

In [8]:
query = "Who you gonna call?"

print(few_shot_prompt_template.format(query = query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: Who you gonna call?
AI: 



The FewShotPromptTemplate thus allows us to use variables within these prompt strings in a highly programmable way. As we'll see below, we can even choose examples based on factors like similarity to the question or length.

In [9]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [10]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 100
)

In [12]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n"
)

print(dynamic_prompt_template.format(query = "Who you gonna call?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: What is the weather like today?
AI: Cloudy with a chance of memes.


User: What is your favorite movie?
AI: Terminator


User: Who is your best friend?
AI: Siri. We have spirited debates about the meaning of life.


User: Who you gonna call?
AI: 



In [14]:
print(openai(dynamic_prompt_template.format(query = "Who you gonna call?")))

Ghostbusters? Is that still relevant?


Haha! As we can see, the bot did get wittier.